<a href="https://colab.research.google.com/github/Aapng-cmd/fish/blob/master/voice_synth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import librosa
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
import torch, librosa
from torch.utils.data import Dataset, DataLoader
# from utils.default_models import ensure_default_models
from torch.nn import functional as F
from torch.optim import AdamW
import os


import torch
import torch.nn as nn
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
NIGGA = 4
print(device)

cpu


In [2]:
from google.colab import files
files.upload()

!mkdir audio
!mkdir data
!mkdir models
!mv sample*.wav audio


Saving sample0.wav to sample0.wav
Saving sample1.wav to sample1.wav
Saving sample2.wav to sample2.wav
Saving sample3.wav to sample3.wav
mkdir: cannot create directory ‘models’: File exists


In [2]:
import speech_recognition as sr
import os
from pydub import AudioSegment
from pydub.silence import split_on_silence
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt')  # download the Punkt tokenizer models

def transcribe_audio(path):
    with sr.AudioFile(path) as source:
        audio_listened = r.record(source)
        try:
            text = r.recognize_google(audio_listened, language="ru-RU")
        except sr.UnknownValueError as e:
            print("Error:", str(e))
            text = ""
        return text

def get_large_audio_transcription_on_silence(path):
    sound = AudioSegment.from_file(path)
    chunks = split_on_silence(sound, min_silence_len=500, silence_thresh=sound.dBFS-14, keep_silence=500)
    folder_name = "audio-chunks"
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    for i, audio_chunk in enumerate(chunks, start=1):
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        text = transcribe_audio(chunk_filename)
        whole_text += text + " "
    sentences = sent_tokenize(whole_text)
    return ". ".join(sentences)

r = sr.Recognizer()
for i in range(NIGGA):
    path = f"audio/sample{i}.wav"
    text = get_large_audio_transcription_on_silence(path)
    with open(f"data/sample{i}.txt", "w") as f:
        f.write(text)

    print(text)


with open('your_dataset.csv', "w") as f:
    f.write('text,audio_file\n')
    for i in range(NIGGA):
        with open(f"data/sample{i}.txt", "r") as f1:
            data = f1.read()
        f.write(data + "," + f"audio/sample{i}.wav" + "\n")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


с доступностью игр в России всё очень-очень сложно тем не менее компания пытается сохранять лояльную аудиторию
вечером ты помнишь вьюга злилась на мутном небе мгла носилась Луна как бледное пятно сквозь тучи мрачные желтела
под голубыми небесами великолепными коврами блестя на Солнце снег лежит
тогда менеджер сообщество под ником дал понять что речь не идёт о какой-то целенаправленной блокировки игроков из России


In [3]:
class AudioDataset(Dataset):
    def __init__(self, audio_folder, transcript_folder):
        self.audio_folder = audio_folder
        self.transcript_folder = transcript_folder
        self.audio_files = [f for f in os.listdir(audio_folder) if f.endswith('.wav')]
        self.transcripts = [f for f in os.listdir(transcript_folder) if f.endswith('.txt')]

        # Create a character set for one-hot encoding
        self.char_set = set()
        for transcript_file in self.transcripts:
            with open(os.path.join(self.transcript_folder, transcript_file), 'r') as f:
                self.char_set.update(f.read())
        self.char_to_index = {char: i for i, char in enumerate(sorted(self.char_set))}

        self.max_audio_size = 213504
        self.max_transcript_length = max(len(transcript) for transcript_file in self.transcripts for transcript in
                                         [open(os.path.join(self.transcript_folder, transcript_file), 'r').read()])

        # Calculate padding to ensure max_audio_size is a multiple of max_transcript_length
        padding_to_multiple = (self.max_transcript_length - (self.max_audio_size % self.max_transcript_length)) % self.max_transcript_length
        self.max_audio_size = self.max_audio_size + padding_to_multiple

    def __getitem__(self, index):
        audio_file = os.path.join(self.audio_folder, self.audio_files[index])
        transcript_file = os.path.join(self.transcript_folder, self.transcripts[index])

        # Check if the transcript file exists
        if index >= len(self.transcripts):
            raise IndexError(f"Index {index} out of range for transcripts")
        if not os.path.exists(transcript_file):
            # Handle the missing transcript
            transcript = ""  # Return an empty string
            audio = torch.empty(self.max_audio_size)
        else:
            audio, sr = librosa.load(audio_file)
            with open(transcript_file, 'r') as f:
                transcript = f.read()

            # Convert the NumPy array to a PyTorch tensor
            audio = torch.from_numpy(audio)

            # Pad the audio tensor to a fixed size
            audio_length = audio.shape[0]
            audio = torch.nn.functional.pad(audio, (0, self.max_audio_size - audio_length))

        # One-hot encode the transcript
        transcript_tensor = torch.zeros(self.max_transcript_length, len(self.char_set))
        for j, char in enumerate(transcript):
            if j < self.max_transcript_length:
                transcript_tensor[j, self.char_to_index[char]] = 1

        # Ensure batch dimension for both audio and transcript
        audio = audio.unsqueeze(0)
        transcript_tensor = transcript_tensor.unsqueeze(0)

        return audio, transcript_tensor

    def __len__(self):
        return len(self.audio_files)

In [4]:
class WaveNet(nn.Module):
    def __init__(self, char_set, receptive_field=2048, dilation_channels=128, residual_channels=128, skip_channels=256):
        super(WaveNet, self).__init__()
        self.char_set = char_set
        self.receptive_field = receptive_field
        self.dilation_channels = dilation_channels
        self.residual_channels = residual_channels
        self.skip_channels = skip_channels

        self.causal_conv = nn.Conv1d(1, dilation_channels, kernel_size=2, padding=1)

        # Dilated convolutional layers
        self.dilated_convs = nn.ModuleList()
        for i in range(1, 10):
            dilation = 2 ** i
            conv = nn.Conv1d(dilation_channels, dilation_channels, kernel_size=2, dilation=dilation, padding=dilation)
            self.dilated_convs.append(conv)

        # Residual and skip connections
        self.residual_convs = nn.ModuleList([nn.Conv1d(dilation_channels, residual_channels, kernel_size=1) for _ in range(9)])
        self.skip_convs = nn.ModuleList([nn.Conv1d(dilation_channels, skip_channels, kernel_size=1) for _ in range(9)])

        # Output layer
        self.output_conv = nn.Conv1d(skip_channels, len(char_set), kernel_size=1)

    def forward(self, x):
        x = torch.relu(self.causal_conv(x))

        skip_connections = []
        for i, (conv, res_conv, skip_conv) in enumerate(zip(self.dilated_convs, self.residual_convs, self.skip_convs)):
            x = torch.relu(conv(x))
            residual = torch.relu(res_conv(x))
            x = x + residual  # Residual connection
            skip_connections.append(torch.relu(skip_conv(x)))

        # Combine skip connections
        skip_output = torch.sum(torch.stack(skip_connections), dim=0)

        # Final output
        output = self.output_conv(skip_output)

        return output

In [5]:
dataset = AudioDataset('audio', 'data')
max_transcript_length = max(len(transcript) for transcript_file in dataset.transcripts for transcript in
                            [open(os.path.join(dataset.transcript_folder, transcript_file), 'r').read()])

dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [6]:
model = WaveNet(dataset.char_set).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(10):
    for batch in dataloader:
        audio_chunks, transcript_chunks = batch

        # Move data to the device
        audio_chunks = audio_chunks.to(device)
        transcript_chunks = transcript_chunks.to(device)

        # Reshape audio chunks to match the expected input shape for Conv1d
        audio_chunks = audio_chunks.view(-1, 1, max_transcript_length)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(audio_chunks)
        loss = criterion(outputs.view(-1, len(dataset.char_set)), transcript_chunks.view(-1))

        # Backward and optimize
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}, Loss: {loss.item()}')